In [1]:
!pip install hazm

  Using cached hazm-0.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached fasttext_wheel-0.9.2-cp312-cp312-win_amd64.whl.metadata (16 kB)
  Using cached flashtext-2.7-py2.py3-none-any.whl
  Using cached gensim-4.3.2.tar.gz (23.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-1.24.3.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  Traceback (most recent call last):
    File "C:\Users\Fateme\Documents\GitHub\NLP\myenv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "C:\Users\Fateme\Documents\GitHub\NLP\myenv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\Fateme\Documents\GitHub\NLP\myenv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
      backend = _build_backend()
                ^^^^^^^^^^^^^^^^
    File "C:\Users\Fateme\Documents\GitHub\NLP\myenv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 77, in _build_backe

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score
from hazm import Normalizer, word_tokenize, stopwords_list, Stemmer

file_path='dataset.xlsx'
df = pd.read_excel(file_path)
df = df[['Content_1', 'Genre']].dropna()

normalizer = Normalizer()
stop_words = set(stopwords_list())
stemmer = Stemmer()

def normalize_text(text):
    text = normalizer.normalize(text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

df['Content_1'] = df['Content_1'].astype(str).fillna('')

# Normalize the content using Hazm
df['Content_1'] = df['Content_1'].apply(normalize_text)

genre_mapping = {
    'Drama': ['Drama', 'Human Interest & Society', 'History', 'Mystery'],
    'Comedy': ['Comedy', 'Arts & Literature', 'Romance', 'Portrait'],
    'Action': ['Action', 'Culture & Traditions', 'Architecture & Urbanism', 'Music'],
    'Crime': ['Crime', 'Family', 'Experimental', 'Nature & Wildlife'],
    'Adventure': ['Adventure', 'War', 'Horror', 'Thriller', 'Animation']
}

# Function to map genres to main genres
def map_genre(genre):
    for main_genre, similar_genres in genre_mapping.items():
        if genre in similar_genres:
            return main_genre
    return None

# Apply the mapping
df['Main_Genre'] = df['Genre'].apply(map_genre)

label_encoder = LabelEncoder()
df['Genre_encoded'] = label_encoder.fit_transform(df['Main_Genre'])

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Genre_encoded'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['Genre_encoded'])

ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(train_df[['Content_1']], train_df['Genre_encoded'])
train_df_resampled = pd.DataFrame({'Content_1': X_train_resampled['Content_1'], 'Genre_encoded': y_train_resampled})

tokenizer = AutoTokenizer.from_pretrained('HooshvareLab/bert-fa-base-uncased')


class PersianMovieGenreDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts.reset_index(drop=True)  # Ensure proper indexing
        self.labels = labels.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

train_dataset = PersianMovieGenreDataset(train_df_resampled['Content_1'], train_df_resampled['Genre_encoded'], tokenizer)
val_dataset = PersianMovieGenreDataset(val_df['Content_1'], val_df['Genre_encoded'], tokenizer)
test_dataset = PersianMovieGenreDataset(test_df['Content_1'], test_df['Genre_encoded'], tokenizer)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = AutoModelForSequenceClassification.from_pretrained('HooshvareLab/bert-fa-base-uncased', num_labels=len(label_encoder.classes_))
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=lambda p: {'accuracy': accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1))}
)

trainer.train()

results = trainer.evaluate(test_dataset)
print(results)

model.save_pretrained('./parsbert_movie_genre_classifier')
tokenizer.save_pretrained('./parsbert_movie_genre_classifier')

predictions = trainer.predict(test_dataset)
y_true = test_df['Genre_encoded'].to_numpy()
y_pred = np.argmax(predictions.predictions, axis=1)

accuracy = accuracy_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_micro = f1_score(y_true, y_pred, average='micro')
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
conf_matrix = confusion_matrix(y_true, y_pred)

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'F1 Score (Macro): {f1_macro}')
print(f'F1 Score (Micro): {f1_micro}')
print(f'Precision (Macro): {precision}')
print(f'Recall (Macro): {recall}')
print('Confusion Matrix:')
print(conf_matrix)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=label_encoder.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

def predict_genre(summary, model, tokenizer, device):
    model.eval()
    inputs = tokenizer(summary, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    with torch.no_grad():
        logits = model(inputs['input_ids'], attention_mask=inputs['attention_mask']).logits
    probs = torch.nn.functional.softmax(logits, dim=-1)
    predicted_class = torch.argmax(probs, dim=-1).item()
    return label_encoder.inverse_transform([predicted_class])[0]

# Example usage

summary = "سالار از طریق چوب بری قاچاق امرار معاش می کند. او یک بار هنگام قطع درخت، دستگیر، اره موتوری اش ضبط و روانه زندان می شود تا دوران محکومیت یک ساله اش را سپری کند. باجی همسر، جلال پسر، ماجان دختر - اعضای خانواده سالار - در غیاب او باید هزینه اره را به صاحب آن میرزا آقا بپردازند. ضمن اینکه ، قبل از واقعه اخیر قرار بوده تا ماجان با لطیف، سرباز اهری پاسگاه روستا ازدواج کند اما بعد از دستگیری سالار، چون باجی، لطیف را هنگام دستگیری شوهرش دیده است، پس در مورد ازدواج لطیف و ماجان تغییر عقیده می دهد. اما لطیف با پیگیری مستمر سعی می کند تا باجی را متقاعد به بی گناهی خود در دستگیری سالار و ازدواج خود با ماجان قانعش کند. جلال که رفتار نامناسب میرزا آقا را با مادرش به خاطر اره ضبط شده، می بیند، با او که رئیس قاچاقچیان چوب جنگل است توافق می کند تا درازای جبران خسارت وی، هر روزه برای فرج و عطا - قاچاقچیان چوب - اره را به میان جنگل ببرد. تحرکات پنهانی و غیبت های مستمر جلال، موجب حساسیت و نگرانی خانواده اش می شود. وقتی ماجان این نگرانی را با لطیف در میان می گذارد، او به تعقیب جلال در یک صبح زود می پردازد، اما لو میرود و توسط قاچاقچیان گرفتار و مورد ضرب و شتم قرار گرفته و جانش را ازدست میدهد. جلال، که در جریان این حادثه به بی گناهی لطیف و خیانت فرج آگاه شده می گریزد. فرج که دریافته او شاهد واقعه بوده، به تعقیبش می پردازد اما او را نمی یابد. باجی و ماجان که از ماوقع آگاه شده اند در جستجوری جلال برمی آیند و او را در درمانگاه روستا با چهره ای سخت پریشان و در هم می یابند، این زمانی است که سالار از زندان آزاد شده و به روستا بازگشته است. او به محض ورود به روستا سراغ لطیف را می گیرد."
predicted_genre = predict_genre(summary, model, tokenizer, torch.device("cuda" if torch.cuda.is_available() else "cpu"))
print(f"Predicted genre: {predicted_genre}")

ModuleNotFoundError: No module named 'hazm'